# Session 2: Data Manipulation and Aggregations

In this session, we'll dive deeper into Polars' data manipulation capabilities. You'll learn how to filter, transform, aggregate, and join data using the powerful expression API.

## Learning Objectives

By the end of this session, you will be able to:
1. Filter rows using boolean expressions
2. Create columns with conditional logic
3. Perform groupby aggregations
4. Handle missing data
5. Join and concatenate DataFrames

In [ ]:
import polars as pl

# Load our datasets
orders = pl.read_csv("data/ecommerce_orders.csv")
customers = pl.read_csv("data/customers.csv")

print(f"Orders: {orders.shape}")
print(f"Customers: {customers.shape}")

In [ ]:
orders.head()

In [ ]:
customers.head()

## 1. Filtering Rows with `filter()`

The `filter()` method lets you select rows based on conditions.

### 1.1 Simple Filters

In [ ]:
# Filter orders over $100
large_orders = orders.filter(pl.col("total_amount") > 100)
print(f"Orders over $100: {large_orders.height}")
large_orders.head()

In [ ]:
# Filter by string equality
electronics = orders.filter(pl.col("category") == "Electronics")
print(f"Electronics orders: {electronics.height}")
electronics.head()

### 1.2 Multiple Conditions

Use `&` (and) and `|` (or) to combine conditions. **Important**: Wrap each condition in parentheses!

In [ ]:
# Electronics orders over $500
big_electronics = orders.filter(
    (pl.col("category") == "Electronics") & 
    (pl.col("total_amount") > 500)
)
print(f"Big electronics orders: {big_electronics.height}")
big_electronics.head()

In [ ]:
# Completed or shipped orders
fulfilled = orders.filter(
    (pl.col("status") == "completed") | 
    (pl.col("status") == "shipped")
)
print(f"Fulfilled orders: {fulfilled.height}")

In [ ]:
# Alternative: use is_in() for multiple values
fulfilled = orders.filter(
    pl.col("status").is_in(["completed", "shipped"])
)
print(f"Fulfilled orders: {fulfilled.height}")

### 1.3 String Filters

In [ ]:
# Products containing "phone" (case-insensitive)
phones = orders.filter(
    pl.col("product_name").str.to_lowercase().str.contains("phone")
)
phones.head()

In [ ]:
# Products starting with "S"
s_products = orders.filter(
    pl.col("product_name").str.starts_with("S")
)
s_products.select("product_name").unique()

### Pandas Comparison: Filtering

| Operation | Pandas | Polars |
|-----------|--------|--------|
| Simple filter | `df[df["col"] > 100]` | `df.filter(pl.col("col") > 100)` |
| Multiple conditions | `df[(df["a"] > 1) & (df["b"] < 5)]` | `df.filter((pl.col("a") > 1) & (pl.col("b") < 5))` |
| Is in list | `df[df["col"].isin(["a", "b"])]` | `df.filter(pl.col("col").is_in(["a", "b"]))` |
| Contains string | `df[df["col"].str.contains("x")]` | `df.filter(pl.col("col").str.contains("x"))` |

## 2. Creating Columns with `with_columns()` and `alias()`

### 2.1 Basic Column Creation

In [ ]:
# Add calculated columns
orders_enhanced = orders.with_columns(
    # Revenue per item
    (pl.col("total_amount") / pl.col("quantity")).alias("revenue_per_item"),
    
    # Discount amount in dollars
    (pl.col("unit_price") * pl.col("quantity") * pl.col("discount")).alias("discount_amount")
)

orders_enhanced.select(
    "product_name", "quantity", "total_amount", "revenue_per_item", "discount_amount"
).head()

### Pandas Comparison: Creating Columns

| Operation | Pandas | Polars |
|-----------|--------|--------|
| Add single column | `df["new"] = df["a"] + df["b"]` | `df.with_columns((pl.col("a") + pl.col("b")).alias("new"))` |
| Add multiple columns | `df.assign(a=..., b=...)` | `df.with_columns(expr1, expr2)` |
| Rename on create | N/A (implicit with assignment) | `.alias("name")` |
| In-place modification | `df["col"] = ...` modifies `df` | `with_columns()` returns new DataFrame |

**Key difference**: Pandas modifies DataFrames in-place by default, while Polars always returns a new DataFrame (immutability).

## 3. Conditional Logic: `when().then().otherwise()`

This is Polars' equivalent of SQL's `CASE WHEN` or Pandas' `np.where()`.

In [ ]:
# Create order size categories
orders_categorized = orders.with_columns(
    pl.when(pl.col("total_amount") >= 500)
      .then(pl.lit("Large"))
      .when(pl.col("total_amount") >= 100)
      .then(pl.lit("Medium"))
      .otherwise(pl.lit("Small"))
      .alias("order_size")
)

orders_categorized.select("product_name", "total_amount", "order_size").head(10)

In [ ]:
# Count by order size
orders_categorized.group_by("order_size").len().sort("len", descending=True)

In [ ]:
# Boolean flag example
orders_flagged = orders.with_columns(
    pl.when(pl.col("status").is_in(["cancelled", "returned"]))
      .then(pl.lit(True))
      .otherwise(pl.lit(False))
      .alias("is_problematic")
)

# Or simpler:
orders_flagged = orders.with_columns(
    pl.col("status").is_in(["cancelled", "returned"]).alias("is_problematic")
)

orders_flagged.select("order_id", "status", "is_problematic").head(10)

### Pandas Comparison

```python
# Pandas with np.where (single condition)
df["is_large"] = np.where(df["amount"] > 500, "Large", "Small")

# Pandas with np.select (multiple conditions)
conditions = [
    df["amount"] >= 500,
    df["amount"] >= 100
]
choices = ["Large", "Medium"]
df["order_size"] = np.select(conditions, choices, default="Small")
```

Polars' `when().then().otherwise()` is more readable and chainable.

## 4. Groupby and Aggregations

The `group_by().agg()` pattern is central to data analysis.

### 4.1 Basic Groupby

In [ ]:
# Total revenue by category
revenue_by_category = orders.group_by("category").agg(
    pl.col("total_amount").sum().alias("total_revenue")
).sort("total_revenue", descending=True)

revenue_by_category

In [ ]:
# Multiple aggregations
category_stats = orders.group_by("category").agg(
    pl.len().alias("order_count"),
    pl.col("total_amount").sum().alias("total_revenue"),
    pl.col("total_amount").mean().alias("avg_order_value"),
    pl.col("quantity").sum().alias("items_sold"),
    pl.col("customer_id").n_unique().alias("unique_customers")
).sort("total_revenue", descending=True)

category_stats

### 4.2 Multiple Grouping Columns

In [ ]:
# Revenue by category and status
category_status = orders.group_by("category", "status").agg(
    pl.len().alias("count"),
    pl.col("total_amount").sum().alias("revenue")
).sort(["category", "revenue"], descending=[False, True])

category_status.head(15)

### 4.3 Common Aggregation Functions

| Function | Description |
|----------|-------------|
| `sum()` | Sum of values |
| `mean()` | Average |
| `median()` | Median |
| `min()` | Minimum |
| `max()` | Maximum |
| `std()` | Standard deviation |
| `var()` | Variance |
| `count()` | Count non-null |
| `len()` | Count all (including null) |
| `n_unique()` | Count unique values |
| `first()` | First value |
| `last()` | Last value |

In [ ]:
# Example: Product performance
product_stats = orders.group_by("product_name").agg(
    pl.len().alias("times_ordered"),
    pl.col("quantity").sum().alias("total_quantity"),
    pl.col("total_amount").sum().alias("total_revenue"),
    pl.col("total_amount").mean().alias("avg_order_value"),
    pl.col("discount").mean().alias("avg_discount")
).sort("total_revenue", descending=True)

product_stats.head(10)

## 5. Missing Data: `fill_null()`, `drop_nulls()`, `is_null()`

Polars uses `null` for missing values (not `NaN` like Pandas).

In [ ]:
# Check for nulls in our data
orders.null_count()

In [ ]:
# Create sample data with nulls
df_with_nulls = pl.DataFrame({
    "name": ["Alice", "Bob", None, "Diana", None],
    "age": [25, None, 35, None, 45],
    "score": [85.0, 90.0, None, 75.0, 88.0]
})
df_with_nulls

In [ ]:
# Count nulls per column
df_with_nulls.null_count()

In [ ]:
# Filter rows where name is null
df_with_nulls.filter(pl.col("name").is_null())

In [ ]:
# Filter rows where name is NOT null
df_with_nulls.filter(pl.col("name").is_not_null())

In [ ]:
# Drop rows with any null
df_with_nulls.drop_nulls()

In [ ]:
# Drop rows with null in specific columns
df_with_nulls.drop_nulls(subset=["name"])

In [ ]:
# Fill nulls with a value
df_filled = df_with_nulls.with_columns(
    pl.col("name").fill_null("Unknown"),
    pl.col("age").fill_null(pl.col("age").mean()),  # Fill with mean
    pl.col("score").fill_null(0.0)
)
df_filled

### Pandas Comparison: Missing Data

| Operation | Pandas | Polars |
|-----------|--------|--------|
| Check null | `df["col"].isna()` | `pl.col("col").is_null()` |
| Drop nulls | `df.dropna()` | `df.drop_nulls()` |
| Fill nulls | `df.fillna(value)` | `df.fill_null(value)` |
| Count nulls | `df.isna().sum()` | `df.null_count()` |

## 6. Sorting with `sort()`

In [ ]:
# Sort by single column (ascending by default)
orders.sort("total_amount").head()

In [ ]:
# Sort descending
orders.sort("total_amount", descending=True).head()

### Pandas Comparison: Sorting

| Operation | Pandas | Polars |
|-----------|--------|--------|
| Sort ascending | `df.sort_values("col")` | `df.sort("col")` |
| Sort descending | `df.sort_values("col", ascending=False)` | `df.sort("col", descending=True)` |
| Multiple columns | `df.sort_values(["a", "b"], ascending=[True, False])` | `df.sort(["a", "b"], descending=[False, True])` |
| Sort by index | `df.sort_index()` | N/A (no index in Polars) |

**Note**: Polars uses `descending` parameter while Pandas uses `ascending`. The logic is inverted!

In [ ]:
# Sort by multiple columns
orders.sort(["category", "total_amount"], descending=[False, True]).head(10)

## 7. Combining DataFrames

### 7.1 Concatenation with `pl.concat()`

### Pandas Comparison: Concatenation

| Operation | Pandas | Polars |
|-----------|--------|--------|
| Vertical (stack rows) | `pd.concat([df1, df2])` | `pl.concat([df1, df2])` |
| Horizontal (add columns) | `pd.concat([df1, df2], axis=1)` | `pl.concat([df1, df2], how="horizontal")` |
| Reset index after concat | `pd.concat(...).reset_index(drop=True)` | N/A (no index in Polars) |

The syntax is similar, but Polars uses `how="horizontal"` instead of `axis=1`.

In [ ]:
# Create two sample DataFrames
df1 = pl.DataFrame({"id": [1, 2], "value": ["a", "b"]})
df2 = pl.DataFrame({"id": [3, 4], "value": ["c", "d"]})

# Vertical concatenation (stack rows)
combined = pl.concat([df1, df2])
combined

In [ ]:
# Horizontal concatenation (add columns)
df3 = pl.DataFrame({"extra": [10, 20]})
pl.concat([df1, df3], how="horizontal")

### 7.2 Joins with `join()`

In [ ]:
# Join orders with customers
orders_with_customers = orders.join(
    customers,
    on="customer_id",
    how="left"
)

print(f"Original orders: {orders.shape}")
print(f"With customer info: {orders_with_customers.shape}")
orders_with_customers.head()

### Pandas Comparison: Joins

| Operation | Pandas | Polars |
|-----------|--------|--------|
| Inner join | `df1.merge(df2, on="key", how="inner")` | `df1.join(df2, on="key", how="inner")` |
| Left join | `df1.merge(df2, on="key", how="left")` | `df1.join(df2, on="key", how="left")` |
| Right join | `df1.merge(df2, on="key", how="right")` | `df1.join(df2, on="key", how="right")` |
| Outer join | `df1.merge(df2, on="key", how="outer")` | `df1.join(df2, on="key", how="full")` |
| Different key names | `df1.merge(df2, left_on="a", right_on="b")` | `df1.join(df2, left_on="a", right_on="b")` |

**Key differences**:
- Pandas uses `merge()`, Polars uses `join()`
- Outer join is `how="outer"` in Pandas, `how="full"` in Polars
- Polars has `semi` and `anti` joins built-in (useful for filtering)

### Join Types

| Type | Description |
|------|-------------|
| `inner` | Only matching rows from both |
| `left` | All rows from left, matching from right |
| `right` | Matching from left, all from right |
| `full` | All rows from both (outer join) |
| `cross` | Cartesian product |
| `semi` | Rows from left that have match in right |
| `anti` | Rows from left that have NO match in right |

In [ ]:
# Analysis: Revenue by customer tier
tier_revenue = orders_with_customers.group_by("membership_tier").agg(
    pl.len().alias("order_count"),
    pl.col("total_amount").sum().alias("total_revenue"),
    pl.col("total_amount").mean().alias("avg_order_value")
).sort("total_revenue", descending=True)

tier_revenue

### Pandas Comparison: String Operations

| Operation | Pandas | Polars |
|-----------|--------|--------|
| Uppercase | `df["col"].str.upper()` | `pl.col("col").str.to_uppercase()` |
| Lowercase | `df["col"].str.lower()` | `pl.col("col").str.to_lowercase()` |
| Length | `df["col"].str.len()` | `pl.col("col").str.len_chars()` |
| Contains | `df["col"].str.contains("x")` | `pl.col("col").str.contains("x")` |
| Replace | `df["col"].str.replace("a", "b")` | `pl.col("col").str.replace("a", "b")` |
| Split | `df["col"].str.split(",")` | `pl.col("col").str.split(",")` |
| Get element from split | `df["col"].str.split(",").str[0]` | `pl.col("col").str.split(",").list.first()` |
| Extract with regex | `df["col"].str.extract(r"(\d+)")` | `pl.col("col").str.extract(r"(\d+)")` |

**Key difference**: After splitting, Pandas uses `.str[n]` indexing while Polars uses `.list.get(n)` or `.list.first()`/`.list.last()`.

In [ ]:
# Analysis: Revenue by customer city
city_revenue = orders_with_customers.group_by("city").agg(
    pl.col("customer_id").n_unique().alias("unique_customers"),
    pl.col("total_amount").sum().alias("total_revenue")
).sort("total_revenue", descending=True)

city_revenue.head(10)

## 8. String Operations

The `.str` namespace provides string manipulation functions.

In [ ]:
# Common string operations
customers_strings = customers.with_columns(
    # Uppercase
    pl.col("first_name").str.to_uppercase().alias("name_upper"),
    
    # Lowercase
    pl.col("city").str.to_lowercase().alias("city_lower"),
    
    # String length
    pl.col("email").str.len_chars().alias("email_length"),
    
    # Extract domain from email
    pl.col("email").str.split("@").list.last().alias("email_domain")
)

customers_strings.select(
    "first_name", "name_upper", "city", "city_lower", "email", "email_length", "email_domain"
).head()

In [ ]:
# String contains and replace
orders.with_columns(
    pl.col("product_name").str.contains("Smart").alias("is_smart_product"),
    pl.col("product_name").str.replace("Smart", "Intelligent").alias("renamed_product")
).filter(pl.col("is_smart_product")).head()

## Summary: Key Methods Covered

| Method | Purpose |
|--------|--------|
| `filter()` | Select rows based on conditions |
| `with_columns()` | Add or modify columns |
| `when().then().otherwise()` | Conditional logic |
| `group_by().agg()` | Groupby aggregations |
| `sort()` | Sort rows |
| `join()` | Combine DataFrames by key |
| `pl.concat()` | Stack DataFrames |
| `drop_nulls()` | Remove rows with nulls |
| `fill_null()` | Replace null values |

## Practice Exercises

Using the `orders` and `customers` DataFrames:

1. Find all cancelled orders with total amount > $200
2. Create a "discount_tier" column: "High" if discount >= 15%, "Low" if > 0%, "None" otherwise
3. Calculate the average order value for each combination of category and status
4. Find the top 5 customers by total spending (join with customers table)

In [ ]:
# Exercise 1


In [ ]:
# Exercise 2


In [ ]:
# Exercise 3


In [ ]:
# Exercise 4


## Next Session Preview

In Session 3, we'll explore Polars' most powerful feature: **Lazy Evaluation**
- Understanding eager vs lazy execution
- Query optimization
- Performance benchmarking
- Working with large datasets